In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import f1_score as f1s
import lightgbm as lgb
import random as r
from sklearn.neural_network import MLPClassifier

# Random seed for reproducibility
r.seed(1)

# Main function: Handles data loading, training, validation, and saving predictions
def train_and_validate(train_data_path, train_labels_path, test_data_path, output_file):

    train_data = np.load(train_data_path, allow_pickle=True).item()  # Training features
    train_labels = pd.read_csv(train_labels_path)  # Training labels
    test_data = np.load(test_data_path, allow_pickle=True).item()  # Test features

    # Combine training features from different sources
    X = np.hstack([
        train_data['vit_feature'],
        train_data['clip_feature'],
        train_data['dino_feature']
    ])
    y = train_labels['label'].values

    # Combine test features from different sources
    X_test = np.hstack([
        test_data['vit_feature'],
        test_data['clip_feature'],
        test_data['dino_feature']
    ])

    print("Combined Training Features:", X.shape)
    print("Combined Test Features:", X_test.shape)

    # 2. Feature Selection (Select top 2000 features)
    print("\nPerforming feature selection...")
    selector = SelectKBest(score_func=f_classif, k=2000)
    X_reduced = selector.fit_transform(X, y)
    X_test_reduced = selector.transform(X_test)  # Apply the same transformation to the test set

    print("Reduced Training Features:", X_reduced.shape)
    print("Reduced Test Features:", X_test_reduced.shape)

    # 3. 5-Fold Cross Validation
    print("\nStarting 5-Fold Cross Validation...")
    kf = KFold(n_splits=5, shuffle=True, random_state=1)  # 5-fold cross-validation
    f1_scores = []

    # Define models
    lightgbm_model = lgb.LGBMClassifier(
        n_estimators=200,
        max_depth=20,
        learning_rate=0.05,
        random_state=1
    )

    mlp_model = MLPClassifier(
        hidden_layer_sizes=(512, 256),  # Two layers: one with 512 neurons, one with 256 neurons
        activation='relu',
        solver='adam',
        max_iter=300,
        random_state=1
    )

    # Cross-validation loop
    for train_index, val_index in kf.split(X_reduced):
        # Create training and validation sets
        X_train, X_val = X_reduced[train_index], X_reduced[val_index]
        y_train, y_val = y[train_index], y[val_index]

        # Train LightGBM model and make predictions
        lightgbm_model.fit(X_train, y_train)
        y_pred_lgb = lightgbm_model.predict_proba(X_val)

        # Train MLP model and make predictions
        mlp_model.fit(X_train, y_train)
        y_pred_mlp = mlp_model.predict_proba(X_val)

        # Combine predictions (weighted average)
        combined_preds = (3 * y_pred_lgb + 3 * y_pred_mlp) / 6
        final_preds = np.argmax(combined_preds, axis=1)

        # Calculate Macro F1 score
        f1 = f1s(y_val, final_preds, average='macro')
        f1_scores.append(f1)
        print(f"Fold F1 Score: {f1:.4f}")

    # Calculate average F1 score
    average_f1 = np.mean(f1_scores)
    print(f"\nAverage F1 Score: {average_f1:.4f}")

    # 4. Make predictions on the test set
    print("\nMaking predictions on the test set...")
    lightgbm_model.fit(X_reduced, y)
    mlp_model.fit(X_reduced, y)

    y_pred_lgb_test = lightgbm_model.predict_proba(X_test_reduced)
    y_pred_mlp_test = mlp_model.predict_proba(X_test_reduced)

    # Combine test predictions
    combined_test_preds = (3 * y_pred_lgb_test + 3 * y_pred_mlp_test) / 6
    final_test_preds = np.argmax(combined_test_preds, axis=1)

    # Save predictions to file
    submission = pd.DataFrame({
        'ID': range(len(final_test_preds)),
        'Predicted': final_test_preds
    })
    submission.to_csv(output_file, index=False)
    print(f"Predictions saved to '{output_file}'")

    return average_f1

# Call the function
average_f1 = train_and_validate("train_feats.npy", "train_labels.csv", "valtest_feats.npy", "final_submission.csv")
